In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv('../data/crawling/everytime_review_data.csv')
df.head(3)

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수,리뷰길이,리뷰_형태소,리뷰_형태소_list,리뷰_형태소_only_N
0,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,22년 2학기 수강자,4,0,28,"[('첫', 'Noun'), ('kcu', 'Alpha'), ('강의', 'Noun...","['첫', 'kcu', '강의', '퀄리티', '떨어지지만', '에', '이쁠', ...","['첫', '강의', '퀄리티', '쉬움']"
1,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,22년 2학기 수강자,4,0,126,"[('그저', 'Adverb'), ('그래요', 'Adjective'), ('다른건...","['그저', '그래요', '다른건', '다', '낫', '배드', '했는데', '기...","['낫', '배드', '기말고사', '서술', '문제', '비중', '제일', '점..."
2,기업가정신과창업 KCU,3,이론,x,전학년,교양,교양,김한수,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 2번 + 기말 시...,22년 2학기 수강자,4,0,226,"[('기', 'Modifier'), ('창', 'Noun'), ('강의', 'Nou...","['기', '창', '강의', '평', '피드백', '없고', '강의', '안도',...","['창', '강의', '평', '피드백', '강의', '안도', '꿀강', '퀴즈'..."


In [ ]:
reviews = df[['리뷰','리뷰_형태소_list']].rename(columns={'리뷰_형태소_list': '형태소'})
reviews['형태소'] = reviews['형태소'].apply(lambda x: re.sub(r"\'", '', x[1:-1]))
reviews.head(3)

,리뷰,형태소
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에, 이쁠, 받기, 쉬움"
1,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술, 형, ..."
2,기창 강의평 피드백 없고 강의안도 없지만 그래도 할만한 꿀강 퀴즈 2번 + 기말 시...,"기, 창, 강의, 평, 피드백, 없고, 강의, 안도, 없지만, 그래도, 할만, 한,..."


## new bigram, trigram

In [ ]:
def NGram(N, DF, ColName, NewColName):
    DF[NewColName] = ''
    for idx in range(len(DF)):
        ColValList = DF._get_value(idx, ColName).split(', ')
        result_list = []
        for val_idx in range(len(ColValList)-(N-1)):
            temp_val = ''.join(ColValList[val_idx:val_idx+N])
            result_list.append(temp_val)
        DF._set_value(idx, NewColName, ', '.join(result_list))
        
NGram(2, reviews, '형태소', '형태소_bi')
NGram(3, reviews, '형태소', '형태소_tri')

In [ ]:
reviews.head(2)

,리뷰,형태소,형태소_bi,형태소_tri
0,첫 kcu 강의 퀄리티 떨어지지만 에이쁠 받기 쉬움,"첫, kcu, 강의, 퀄리티, 떨어지지만, 에, 이쁠, 받기, 쉬움","첫kcu, kcu강의, 강의퀄리티, 퀄리티떨어지지만, 떨어지지만에, 에이쁠, 이쁠받...","첫kcu강의, kcu강의퀄리티, 강의퀄리티떨어지지만, 퀄리티떨어지지만에, 떨어지지만..."
1,그저그래요 다른건 다 낫배드했는데 기말고사로는 서술형 1문제가 나왔어요 비중도 제일...,"그저, 그래요, 다른건, 다, 낫, 배드, 했는데, 기말고사, 로는, 서술, 형, ...","그저그래요, 그래요다른건, 다른건다, 다낫, 낫배드, 배드했는데, 했는데기말고사, ...","그저그래요다른건, 그래요다른건다, 다른건다낫, 다낫배드, 낫배드했는데, 배드했는데기..."


## 개수 분석

In [ ]:
#경고제어
import warnings
warnings.simplefilter("ignore") # 일치하는 경고 인쇄 x, simplefilter : 경고 필터 명세 목록에 간단한 항목을 삽입

#텍스트 분석에 필요한 konlpy 불러오기 및 버전 확인
import konlpy
konlpy.__version__

#KoNLPy의 OKT(형태소 분석기) 불러오기
from konlpy.tag import Okt

import nltk
from konlpy.tag import Okt

# 여기 수행시간 좀 걸림
from collections import Counter
import collections

In [ ]:
def NGramCounts(DF, ColName):
    result_list = []
    for idx in range(len(DF)):
        temp_val = DF._get_value(idx, ColName).split(', ')
        result_list.append(temp_val)
    result = Counter(sum(result_list, []))
    result_df = pd.DataFrame.from_dict(result, orient='index').reset_index().rename(columns={'index': '단어', 0: '갯수'}).sort_values(by='갯수', ascending=False).reset_index(drop=True)
    return result_df

# 바이그램
df_bi = NGramCounts(reviews, '형태소_bi')
# 트라이그램
df_tri = NGramCounts(reviews, '형태소_tri')

In [ ]:
df_bi.to_csv('../data/바이그램단어.csv', encoding='utf-8-sig', index=False)
df_tri.to_csv('../data/트라이그램단어.csv', encoding='utf-8-sig', index=False)